# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [27]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 

'gdown' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
unzip:  cannot find either data.zip or data.zip.zip.
'ls' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [28]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [29]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        # X为numpy数组，而y为标签，没有提供y则默认为None
        self.data = torch.from_numpy(X).float()  # 将数据转变为torch内的张量，并且转为float型
        if y is not None:
            # 若提供y，则将其转换为PyTorch中的长整型张量
            y = y.astype(np.int32)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        # 这个方法是通过索引idx获取数据集中的一个样本
        if self.label is not None:
            # 标签也存在则返回数据和对应标签
            return self.data[idx], self.label[idx]
        else:
            # 否则只返回数据
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [30]:
VAL_RATIO = 0.2  # 这里是划分交叉验证集和训练数据集的比例的地方，是个hyperparameter

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y = train[:percent], train_label[:percent]
val_x, val_y = train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [31]:
BATCH_SIZE = 2048  # 这是设置批大小的地方，这里可以自由调整

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [32]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

580

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [33]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer0 = nn.Linear(429, 2048)
        self.bn0 = nn.BatchNorm1d(2048)  # 这里加入了batch normalization的优化尝试，对每层均进行
        self.layer1 = nn.Linear(2048, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.layer2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.layer3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.layer4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.layer5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        self.out = nn.Linear(64, 39) 

        self.act_fn = nn.ReLU()
        self.drop = nn.Dropout(0.5)

    def forward(self, x):
        x = self.layer0(x)
        x = self.bn0(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer4(x)
        x = self.bn4(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer5(x)
        x = self.bn5(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.out(x)
        
        return x

## Training

In [34]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [35]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [36]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters, 这一块可以自己调整更改
num_epoch = 100               # number of training epoch
learning_rate = 0.001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

DEVICE: cuda


In [37]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):  # 对于每一个批次
        inputs, labels = data  # 获取本次数据和标签
        inputs, labels = inputs.to(device), labels.to(device)  # 根据前面获取的device设置，放入GPU或CPU
        optimizer.zero_grad()  # 初始梯度清零，防止梯度累积
        outputs = model(inputs)  # 前向传播，获得结果outputs
        batch_loss = criterion(outputs, labels)  # 计算当前批次的损失
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability，其实就是获取预测的类别
        batch_loss.backward()  # 反向传播计算梯度
        optimizer.step()  # 更新学习率

        # 计算训练时的准确率和损失
        # 准确率的计算方式是：累加预测的类别和实际标签相同的个数
        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():  # 禁止梯度计算，以节省内存和计算时间
            for i, data in enumerate(val_loader):  # 同样对于每个批次，大体和训练时相同，只是不需要计算梯度和更新参数
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            # 打印出结果以查看训练情况，实际上前面计算的都是预测正确的数量，这里打印准确率采用直接除法得出结果
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:  # 只有模型性能更好时保存参数
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        # 如果没有验证集，则直接输出训练集结果
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/100] Train Acc: 0.402867 Loss: 2.151267 | Val Acc: 0.544919 loss: 1.504248
saving model with acc 0.545
[002/100] Train Acc: 0.505922 Loss: 1.721443 | Val Acc: 0.602072 loss: 1.330331
saving model with acc 0.602
[003/100] Train Acc: 0.542908 Loss: 1.609000 | Val Acc: 0.636615 loss: 1.236910
saving model with acc 0.637
[004/100] Train Acc: 0.566218 Loss: 1.540344 | Val Acc: 0.655994 loss: 1.172505
saving model with acc 0.656
[005/100] Train Acc: 0.583814 Loss: 1.486284 | Val Acc: 0.671161 loss: 1.131601
saving model with acc 0.671
[006/100] Train Acc: 0.598720 Loss: 1.442010 | Val Acc: 0.682443 loss: 1.095370
saving model with acc 0.682
[007/100] Train Acc: 0.608711 Loss: 1.408498 | Val Acc: 0.690728 loss: 1.068428
saving model with acc 0.691
[008/100] Train Acc: 0.617685 Loss: 1.380134 | Val Acc: 0.695390 loss: 1.050893
saving model with acc 0.695
[009/100] Train Acc: 0.625185 Loss: 1.355061 | Val Acc: 0.701053 loss: 1.028821
saving model with acc 0.701
[010/100] Train Acc: 0.63109

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [38]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

C:\Users\BQ\AppData\Local\Temp\ipykernel_14476\2373171232.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

Make prediction.

In [39]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [40]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))